In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install --upgrade pip
%pip install seaborn
%pip install keras
%pip install tenserflow

In [ ]:
import numpy as np
import os
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout

import tensorflow as tf
from keras.optimizers import Adam


In [ ]:
data_dir = "./drive/MyDrive/DataSet1/" 

general_lables = ['Defected', 'OK']
img_size = 128 # image size
def get_data(data_dir):
    images = []
    labels = []
    for label in general_lables:
        path = os.path.join(data_dir, label)
        class_num = general_lables.index(label)
        for img in os.listdir(path):
            try:
                f = os.path.join(path, img)
                img_arr = cv2.imread(f, 0)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                images.append(resized_arr)
                labels.append(class_num)
            except Exception as e:
                print(e)
    return np.array(images), np.array(labels)

images, labels = get_data(data_dir)


In [ ]:
num_samples = len(images)
num_trains = int((num_samples * 3) / 4) # 3/4 of data used for training and 1/3 for validation(testing)
train_images = images[:num_trains]
train_labels = labels[:num_trains]
test_images = images[num_trains:]
test_labels = labels[num_trains:]

In [ ]:
l = [general_lables[i] for i in train_labels]
sns.set_style('darkgrid')
sns.countplot(l)

In [ ]:
# plot random images
plt.figure(figsize = (5,5))
plt.imshow(train_images[4], cmap='gray')
plt.title(general_lables[train_labels[4]])

In [ ]:
# plot random images
plt.figure(figsize = (5,5))
plt.imshow(train_images[-1], cmap='gray')
plt.title(general_lables[train_labels[-1]])

In [ ]:
x_train = train_images
y_train = train_labels
x_val = test_images
y_val = test_labels

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)


In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [ ]:
datagen.fit(x_train)


In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(128,128,1)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [ ]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

In [ ]:
# training the model
history = model.fit(x_train,y_train,epochs = 20 , validation_data = (x_val, y_val), callbacks=[cp_callback])

In [ ]:
# saving the model
model.save('binary_classification_model.keras')

In [ ]:
# just run a test
img = cv2.imread("./drive/MyDrive/DataSet1/OK/aug_gauss_noise_cast_ok_0_1234.jpeg", 0)
img = cv2.resize(img, (128, 128))
img = np.array(img)
img = img.reshape(-1, 128, 128, 1)
img.shape
y = model.predict(img)
general_lables[np.argmax(y)]